In [4]:
from os import listdir
from os.path import isfile, join
import h5py
import cv2
import numpy as np
from skimage.util import view_as_blocks
from time import gmtime, strftime
from multiprocessing import Pool

def Down_Sample(image, block_size, func=np.sum, cval=0):

    if len(block_size) != image.ndim:
        raise ValueError("`block_size` must have the same length "
                         "as `image.shape`.")

    pad_width = []
    for i in range(len(block_size)):
        if block_size[i] < 1:
            raise ValueError("Down-sampling factors must be >= 1. Use "
                             "`skimage.transform.resize` to up-sample an "
                             "image.")
        if image.shape[i] % block_size[i] != 0:
            after_width = block_size[i] - (image.shape[i] % block_size[i])
        else:
            after_width = 0
        pad_width.append((0, after_width))

    image = np.pad(image, pad_width=pad_width, mode='constant',
                   constant_values=cval)

    out = view_as_blocks(image, block_size)

    for i in range(len(out.shape) // 2):
        out = func(out, axis=-1)

    return out

def RmIsolated_pixel(img):

    sat = np.pad(img, pad_width=1, mode='constant', constant_values=0)
    sat = np.cumsum(np.cumsum(sat, axis=0), axis=1)
    sat = np.pad(sat, ((1, 0), (1, 0)), mode='constant', constant_values=0)
    # These are all the possible overlapping 3x3 windows sums
    sum3x3 = sat[3:, 3:] + sat[:-3, :-3] - sat[3:, :-3] - sat[:-3, 3:]
    # This takes away the central pixel value
    sum3x3 -= img
    # This zeros all the isolated pixels
    img[sum3x3 == 0] = 0

    return img

def FileProcess(filename):

    print 'processing '+filename
    data=h5py.File(filename)
    img=np.asarray(data['savedata'])
    img=img.T
    imgB=img[:,:,2]
    img=img[:,:,1] #take only green channel
    #imgB=img[:,:,2]
    
    img=img*255 #make the graph binary
    imgB=imgB*255
    
    img=RmIsolated_pixel(img)
    imgB=RmIsolated_pixel(imgB)
    
    for i in range(0,3):
        img=Down_Sample(img,block_size=(2,2),func=np.sum)
        imgB=Down_Sample(imgB,block_size=(2,2),func=np.sum)
    img=img*255
    imgB=imgB*255
    cv2.imwrite(filename.split('.')[0]+'-Green.jpg',img)
    cv2.imwrite(filename.split('.')[0]+'-Blue.jpg',imgB)

    print 'finish '+filename




In [2]:
filepath='/home/xli/Documents/fluoroonly/'
filelist = [filepath + f for f in listdir(filepath) if isfile(join(filepath, f))]

In [3]:
type(filelist)

list

In [5]:
p=Pool(2)
p.map(FileProcess,tuple(filelist))

processing /home/xli/Documents/fluoroonly/M820-F80--_3_0240_fluoroonly.mat
processing /home/xli/Documents/fluoroonly/M820-F33--_2_0098_fluoroonly.mat
finish /home/xli/Documents/fluoroonly/M820-F33--_2_0098_fluoroonly.mat
processing /home/xli/Documents/fluoroonly/M820-F150--_3_0450_fluoroonly.mat
finish /home/xli/Documents/fluoroonly/M820-F150--_3_0450_fluoroonly.mat
processing /home/xli/Documents/fluoroonly/M820-F28--_3_0084_fluoroonly.mat
finish /home/xli/Documents/fluoroonly/M820-F80--_3_0240_fluoroonly.mat
processing /home/xli/Documents/fluoroonly/M820-F39--_2_0116_fluoroonly.mat
finish /home/xli/Documents/fluoroonly/M820-F28--_3_0084_fluoroonly.mat
processing /home/xli/Documents/fluoroonly/M820-F178--_3_0534_fluoroonly.mat
finish /home/xli/Documents/fluoroonly/M820-F39--_2_0116_fluoroonly.mat
processing /home/xli/Documents/fluoroonly/M820-F91--_3_0273_fluoroonly.mat
finish /home/xli/Documents/fluoroonly/M820-F178--_3_0534_fluoroonly.mat
processing /home/xli/Documents/fluoroonly/M82

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [4]:
s=filelist[1]

In [21]:
data=h5py.File(s)
img=np.asarray(data['savedata'])
img=img.T
img=img[:,:,2] #take only green channel
img=img*255 #make the graph binary

In [20]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())
img=RmIsolated_pixel(img)
print strftime("%Y-%m-%d %H:%M:%S", gmtime())

2017-05-02 01:24:56
2017-05-02 01:30:10


In [8]:
img.shape

(36000, 48000)

In [22]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())
for i in range(0,3):
    img=Down_Sample(img,block_size=(2,2),func=np.sum)
print strftime("%Y-%m-%d %H:%M:%S", gmtime())


2017-05-02 02:07:37
2017-05-02 02:08:08


In [10]:
img.shape

(4500, 6000)

In [23]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())
img=img*255
cv2.imwrite(s.split('.')[0]+'-blue.jpg',img)
print strftime("%Y-%m-%d %H:%M:%S", gmtime())


2017-05-02 02:08:08
2017-05-02 02:08:09


In [12]:
s

'm820F/fluoroonly/M820-F196--_3_0588_fluoroonly.mat'

In [17]:
s.split('.')[0]

'm820F/fluoroonly/M820-F196--_3_0588_fluoroonly'